In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import ipywidgets as widgets
from ipywidgets import Layout

# Function to calculate the next power of 2 for FFT optimization
def nextpow2(i):
    n = 1
    while n < i: 
        n *= 2
    return n

# Custom implementation of pwelch function for power spectral density estimation
def pwelch(x, Fs):                    
    Ts = 1 / Fs  # Sampling period
    L = np.size(x) + 1  # Signal length
    T = L * Ts  # Signal duration
    N = 2^nextpow2(L)  # Next power of 2 for FFT
    Fo = Fs / N  # Frequency resolution
    f = np.arange(0, N) * Fo  # Frequency vector
        
    # Determine the window size and the number of windows
    window_size = nextpow2(np.size(x) / 8)
    if window_size < 256:
        window_size = 256
    windows = np.size(x) // (window_size // 2) - 1
    indexer = np.arange(window_size)[None, :] + (window_size // 2) * np.arange(windows)[:, None]
    windowed_x = x[indexer]

    avg_pwr = 0
    # Process each window
    for window in windowed_x:
        window = window * np.hanning(np.size(window))  # Apply a Hanning window
        L = np.size(window) + 1                 
        T = L * Ts                     
        N = 2^nextpow2(L)
        Fo = Fs / N                   
        f = np.arange(0, N) * Fo
        window_fft = np.fft.fft(window, N)  # FFT of the window
        power = np.multiply(window_fft, np.conj(window_fft)) / N / L  # Power calculation
        avg_pwr += power  # Accumulate power across windows
    
    avg_pwr /= windows  # Average power over windows

    return f[np.arange(0, N // 2)], avg_pwr[np.arange(0, N // 2)]  # Return frequency and power

# Function to update plots based on the slider value for sampling frequency
def update_plots(sampling_frequency):
    signal_length = 1000  # Signal length
    sample_period = 1 / sampling_frequency  # Update sampling period based on slider
    time_vector = np.arange(0, signal_length) * sample_period  # Generate time vector
    
    # Create a composite signal with different frequencies
    signal_data = np.sin(2 * np.pi * 30 * time_vector) + 0.8 * np.sin(2 * np.pi * 80 * (time_vector - 2)) + np.sin(2 * np.pi * 60 * time_vector)
    
    # Compute custom pwelch
    custom_frequencies, custom_power = pwelch(signal_data, sampling_frequency)
    
    # Compute signal.welch using SciPy
    welch_frequencies, welch_power = signal.welch(signal_data, fs=sampling_frequency)
    
    # Compute FFT for the signal
    L = len(signal_data)  # Signal length
    N = 1 * L  # FFT length
    Fo = sampling_frequency / N  # Frequency resolution
    Fx = np.fft.fft(signal_data, N)  # Compute FFT
    freq = np.arange(0, N) * Fo  # Frequency vector
    
    # Compute periodogram
    power = ((Fx * np.conj(Fx)) / (sampling_frequency * L)).real  # Power spectral density

    # Plot the results
    fig, axs = plt.subplots(4, 1, figsize=(15, 20))
    
    # Plot FFT result
    axs[0].plot(freq, np.abs(Fx))
    axs[0].set(xlabel='Frequency (Hz)', ylabel='Magnitude', title='FFT of the Signal')
    axs[0].grid()

    # Plot periodogram
    axs[1].plot(freq, power)
    axs[1].set(xlabel='Frequency (Hz)', ylabel='Power', title='Periodogram')
    axs[1].grid()

    # Plot custom pwelch result
    axs[2].plot(custom_frequencies, custom_power)
    axs[2].set(xlabel='Frequency (Hz)', ylabel='Power', title='Periodogram (Custom pwelch)')
    axs[2].grid()
    
    # Plot signal.welch result from SciPy
    axs[3].plot(welch_frequencies, welch_power)
    axs[3].set(xlabel='Frequency (Hz)', ylabel='Power', title='Periodogram (signal.welch)')
    axs[3].grid()

    plt.tight_layout()  # Adjust layout for better fit
    plt.show()

# Create a slider for adjusting the sampling frequency
sampling_frequency_slider = widgets.IntSlider(
    value=500,  # Initial value
    min=100,  # Minimum frequency
    max=2000,  # Maximum frequency
    step=100,  # Step size
    description='Sampling Frequency (Fs):',
    layout=Layout(width='auto', flex='1 1 auto'),
    style={'description_width': 'initial'}, 
    continuous_update=False  # Update plot only after slider adjustment
)

# Layout for slider and output
vbox_layout = Layout(display='flex', flex_flow='column', align_items='center')

# Group the slider input
inputs_box = widgets.HBox([sampling_frequency_slider], layout=Layout(flex='1 1 auto', width='auto'))

# Create interactive interface
ui = widgets.HBox([inputs_box], layout=Layout(display='flex', justify_content='center', width='100%', align_items='center'))

# Connect slider to plot update function
output = widgets.interactive_output(update_plots, {'sampling_frequency': sampling_frequency_slider})

# Display UI and output
display(ui, output)


Output()